# Exploring parallelism and concurrency in Python

__Elliott Forney - 2020__

---

Building software that leverages concurrency and parallelism in Python can be a sophisticated topic.  Furthermore, there are many common misunderstandings about how and the degree to which parallelism can be achieved in Python.  In this talk, we will discuss some of the intricacies of concurrency and parallelism along with simple examples of how to use async, multithreaded and multiprocessing paradigms.  We will also briefly discuss CPython's Global Interpreter Lock (GIL) and we will dispell some of the common misconceptions about how and when it limits parallelism in Python.

To summarize, we will examine:

1.  Differences between parallelism and concurrency
1.  CPython Global Interpreter Lock
1.  A sequential toy problem
1.  Concurrency with asynchronous programming and coroutines
1.  Concurrency and parallelism with threads
1.  Concurrency and parallelism with multiprocessing
1.  Concurrency and parallelism in C extensions

## Parallelism and Concurrency

---

### True or False:  "Parallelism and concurrency are different words for the same thing."

__False__:  Parallelism indicates that multiple processes or threads of execution are running *at the same time*, e.g., on multiple cores, CPUs or machines.

The term "concurrent," on the other hand, simply implies that things are happening at the same time.  It is possible to have concurrency *with or without* parallelism.  Consider, for example, that when running multiple processes or threads of execution on a machine with a single core, the operating system may *context switch* between threads of execution, giving the *illusion* of parallelism, but not necessarily the speedup.  Think multiple GUIs running on a single-core machine.  In this case, we have concurrency but not parallelism.  There are also other ways to achieve concurrency, including the use of coroutines, which we will examine shortly.  If, however, we have multiple processes or threads running independently on separate CPU cores, we may have *both* concurrency and parallelism.

## The infamous Global Interpreter Lock (GIL)

---

### True or False:  "Python does not have real multithreading because of the GIL."

__False__:  Not only does python have threads, they are true operating system level threads and can provide both parallelism and concurrency.  In the CPython implementation of Python (which is the most common) there is a Global Interpreter Lock (GIL) which **prevents multiple threads from holding access to the interpreter at the same time**.

There are many situations, however, where the GIL is actually freed and multiple threads are permitted to run simultanuously in Python.  Again, we will see examples of this shortly.

---

### True or False:  "The only way to achieve concurrency or parallelism in Python is to fork multiple process."

__False__:  One way to overcome the limitations of the GIL is to fork multiple Python interpreters that each have their own GIL.  We will demonstrate, however, that this is __far from the only way__ to achieve concurrency or parallelism in Python, depending on your application.

### Quick questions,  when would the audience

1.  Use async in Python
1.  Use threads in Python
1.  Use multiprocessing in Python

## A toy problem

---

In order to begin exploring concurrency and parallelism in Python, let's first construct a toy problem that may have several opportunities for leveraging concurrency and parallelism.

For these examples, we will leverage NumPy to create some computationally expensive calls.  We will also leverage the `time` module in order to intentionally introduce some blocking sleeps and to time the results of our solutions.

In [1]:
import numpy as np
import time

Say that we have an expensive function call that services some sort of requests.  In a microservice world, each call to this function might correspond to a separate call to our microservice.

The `service_request` function below is designed to simulate such a request.  This function does several things and, from the perspective of concurrency and parallelism, it is important that we understand each of these steps.

1.  First, note that `service_request` performs five iterations.  This may correspond to an iterative, numerical algorithm that must perform a number of iterations in order to converge.  This also helps to increase the computational requirements of this toy problem and to help us examine when concurrent execution is occuring.
1.  Second, note that there is a blocking call to `time.sleep` in each iteration that lasts for a random amount of time in [0, 1] seconds.  This simulates a __blocking__ function call, which is similar to an I/O request that is reading from disk or, in the microservice world, this simulates a request to a downstream service.
1.  Third, each iteration computes the Fast Fourier Transform of a random array of floating point values.  This simulates an action that is computationally expensive, from a numerical standpoint.  Note that the NumPy `fft` function is implemented in C under the hood, but runs in a single thread (more on this later).

In [2]:
def service_request(i):
    '''This is a toy function that involves both a blocking sleep
    operation and a single-threaded computational, FFT operation.'''
    # perform five iterations
    for j in range(5):
        # log that the iteration has begun
        print(f'call {i} iteration {j}')

        # perform a blocking sleep call ~ [0, 1] seconds
        # this simulates an I/O operation or network request
        secs = np.random.random()
        time.sleep(secs)

        # and then we do some heavy CPU computation
        # note: that the NumPy FFT uses only a single core
        s = np.random.random(4000000)
        z = np.fft.fft(s)

        # log that the iteration is complete
        print(f'end {i} iteration {j}')

Let's also create a simple decorated called `@timed` that allows us to report the amount of time that a function call takes, without having to rewrite the code.  If you haven't seen a custom python decorator before, consider reviewing this topic!  They can be very useful.  The details of decorators is not, however, important for the remaining examples, other than to realize that this decorator will print the time taken by a function call in seconds.

In [3]:
import functools

def timed(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        total_time = time.time() - start_time
        print('\x1b[31mtook:\x1b[0m', np.round(total_time, decimals=4))
        return result
    return wrapper

## Sequential Execution

---

The simpliest way for us to simulate a number of requests is to simply run them sequentially or consecutively from a simple for loop.  For reference, the following example does just that and reports back the total time taken.

In [4]:
@timed
def run_sequential(n=10):
    '''Run `service_request` sequentially `n` times.'''
    for i in range(n):
        service_request(i)
    
run_sequential(10)

call 0 iteration 0
end 0 iteration 0
call 0 iteration 1
end 0 iteration 1
call 0 iteration 2
end 0 iteration 2
call 0 iteration 3
end 0 iteration 3
call 0 iteration 4
end 0 iteration 4
call 1 iteration 0
end 1 iteration 0
call 1 iteration 1
end 1 iteration 1
call 1 iteration 2
end 1 iteration 2
call 1 iteration 3
end 1 iteration 3
call 1 iteration 4
end 1 iteration 4
call 2 iteration 0
end 2 iteration 0
call 2 iteration 1
end 2 iteration 1
call 2 iteration 2
end 2 iteration 2
call 2 iteration 3
end 2 iteration 3
call 2 iteration 4
end 2 iteration 4
call 3 iteration 0
end 3 iteration 0
call 3 iteration 1
end 3 iteration 1
call 3 iteration 2
end 3 iteration 2
call 3 iteration 3
end 3 iteration 3
call 3 iteration 4
end 3 iteration 4
call 4 iteration 0
end 4 iteration 0
call 4 iteration 1
end 4 iteration 1
call 4 iteration 2
end 4 iteration 2
call 4 iteration 3
end 4 iteration 3
call 4 iteration 4
end 4 iteration 4
call 5 iteration 0
end 5 iteration 0
call 5 iteration 1
end 5 iteration 1
c

## Asynchronous programming with `asyncio` and coroutines

---

* Advantage: Provides concurrency and keeps CPU busy during I/O-bound operations.
* Disadvantage:  Does not provide CPU parallelism.

One way to achieve a performance improvement for this toy problem is to use asynchronous programming.  Asynchronous programming, which was widely popularized by JavaScript, is now natively supported in Python.  The gist of async programming is that we can have multiple functions that are defined with the `async` keyword.  When these functions are called, they return what is called a `future` (similar to a `promise` in JavaScript).  Rather than running executing immediately, we can schedule these `future`s to be run by an event scheduler.  The event scheduler will pick up the next future in its queue and see if it is available to run.  If it is ready to run, it will execute until it yields control back.  If it is not ready to run or when it yields control back to the scheduler, the next available future will be examined.  In this way, we can have many functions, called `coroutines`, running in an interleaved fashion.  This allows us to achieve a level of concurrency.  Async programming does not provide CPU parallelism because only one coroutine is executing on the CPU at any given time.  It can improve performance, however, if a coroutine yields back execution while it is waiting for blocking requests like I/O.

In order for an `async` function to yield control back to the scheduler, the `await` keyword must be used (which tells the function to wait for another `future` to complete executing, on another `async` function.  This is best demonstrated by example.  Below, we have an `async` version of our previous `service_request` function.  In addition to adding the `async` keyword to tell Python that this function returns a `future`, we also use `await asynchio.sleep` instead of `time.sleep`.  At this point in the execution of our coroutine, execution will be yielded back to the event scheduler and another coroutine can run the computational portion (the `np.fft`) while another coroutine is sleeping.

In the microservice world, this means that we can continue to service requests and to utilize our CPU, even while waiting for a downstream request or while waiting for a read or write operation from storage.

In [5]:
# many useful async tools are provided by the `asyncio` module
import asyncio

async def service_request_async(i):
    '''This function is the same as `service_request` except that
    it supports the asynchronous programming in order to allow
    other coroutines to run while it is blocking.'''
    for j in range(5):
        print(f'call {i} iteration {j}')

        # now we use `asyncio.sleep` instead of `time.sleep`.
        # this sleep function is `async` so that we can yield
        # control of the interpreter to another function while
        # it is blocking
        secs = np.random.random()
        await asyncio.sleep(secs)

        # nothing different here
        # sice our computation is CPU-bound, i.e., not blocking
        # we have nothing to gain from making it `async` because
        # we cannot yield control while we wait
        s = np.random.random(4000000)
        z = np.fft.fft(s)

        print(f'end {i} iteration {j}')

In order to execure our async function, we must gather together the futures for all of our calls and then `await` for them to complete.

Note that Jupyter notebooks already have an async loop running, so this is all that is necessary.  If this were an ordinary Python program, we would also have to create an event loop and tell it to run until all scheduled futures have completed.

Also note that our `@timed` decorator doesn't work in this case because the function immediately returns the futures.

In [6]:
def run_async(n=10):
    '''Run our toy problem using the async paradigm.'''
    return asyncio.gather(*[service_request_async(i) for i in range(10)])

start_time = time.time()

await run_async();

total_time = time.time() - start_time
print('\x1b[31mtook:\x1b[0m', np.round(total_time, decimals=4))

call 0 iteration 0
call 1 iteration 0
call 2 iteration 0
call 3 iteration 0
call 4 iteration 0
call 5 iteration 0
call 6 iteration 0
call 7 iteration 0
call 8 iteration 0
call 9 iteration 0
end 0 iteration 0
call 0 iteration 1
end 8 iteration 0
call 8 iteration 1
end 0 iteration 1
call 0 iteration 2
end 4 iteration 0
call 4 iteration 1
end 7 iteration 0
call 7 iteration 1
end 5 iteration 0
call 5 iteration 1
end 9 iteration 0
call 9 iteration 1
end 1 iteration 0
call 1 iteration 1
end 2 iteration 0
call 2 iteration 1
end 6 iteration 0
call 6 iteration 1
end 3 iteration 0
call 3 iteration 1
end 7 iteration 1
call 7 iteration 2
end 8 iteration 1
call 8 iteration 2
end 0 iteration 2
call 0 iteration 3
end 4 iteration 1
call 4 iteration 2
end 1 iteration 1
call 1 iteration 2
end 5 iteration 1
call 5 iteration 2
end 9 iteration 1
call 9 iteration 2
end 3 iteration 1
call 3 iteration 2
end 6 iteration 1
call 6 iteration 2
end 2 iteration 1
call 2 iteration 2
end 7 iteration 2
call 7 iteratio

We can see that these requests are now service concurrently and also that the total run time is much better than our sequential example because the CPU can be kept busy during each sleep!

Examining `top` while this code executes reveals, however, that only a single CPU is used at a given time.  In other words, we have achieved a level of concurrency, but __not__ parallelism.

## Concurrency and parallelism with threads

---

* Advantage: shared memory, just like regular threads in C, C++ or Java
* Disadvantage: GIL requires that only one thread can access the interpreter at once

Python also provides the ability to create threads.  Threads have **shared memory** so that if a variable is changed in one thread, it is also changed in another.  Threads can be executed independently which intrinsically allows them to achieve concurrency.  Threads can also leverage multiple CPUs or CPU cores.

Now, as we mentioned before, CPython has a GIL which prevents multiple threads from accessing the interpreter at the same time.  There are, however, many cases when the GIL is actually *released* and multiple threads are allowed to execute in parallel.  Before discussing this further, let's set up a demonstration of our `service_request` example run inside a pool of threads.

In [7]:
# the threading module provides thread-related tools
import threading

@timed
def run_threads(n=10):
    # create threads using the previous, nonasync `service_request` function
    threads = [threading.Thread(target=service_request, args=(i,)) for i in range(10)]

    # start them all
    for thread in threads:
        thread.start()

    # wait for them all to finish
    for thread in threads:
        thread.join()

run_threads()

call 0 iteration 0
call 1 iteration 0
call 2 iteration 0
call 3 iteration 0
call 4 iteration 0
call 5 iteration 0
call 6 iteration 0
call 7 iteration 0
call 8 iteration 0
call 9 iteration 0
end 5 iteration 0
call 5 iteration 1
end 4 iteration 0
call 4 iteration 1
end 0 iteration 0
call 0 iteration 1
end 8 iteration 0
call 8 iteration 1
end 6 iteration 0
call 6 iteration 1
end 2 iteration 0
call 2 iteration 1
end 1 iteration 0
call 1 iteration 1
end 7 iteration 0
call 7 iteration 1
end 9 iteration 0
call 9 iteration 1
end 5 iteration 1
call 5 iteration 2
end 4 iteration 1
call 4 iteration 2
end 3 iteration 0
call 3 iteration 1
end 6 iteration 1
call 6 iteration 2
end 0 iteration 1
call 0 iteration 2
end 8 iteration 1
call 8 iteration 2
end 2 iteration 1
call 2 iteration 2
end 1 iteration 1
call 1 iteration 2
end 9 iteration 1
call 9 iteration 2
end 3 iteration 1
call 3 iteration 2
end 4 iteration 2
call 4 iteration 3
end 7 iteration 1
call 7 iteration 2
end 6 iteration 2end 1 iteration 

Notice that our threads execute in an interleaved fashion, which we would expect from concurrency, but also that the run time is much less than even our async implementation!

How is this possible since the GIL is supposed to prevent multiple threads from running in parallel?!

The answer is that the GIL is released during both the `time.time` and also during the `np.fft` calls, which allows these sections of code to run in parallel on multiple CPUs.  Examining `top` while this code executes will demonstrate that multiple cores are, in fact, in use.

As it turns out, many functions in Python that block on I/O and network traffic actually release the GIL while they wait.  Also, C extensions can release the GIL while they are performing computations that do not require access to the interpreter, e.g., `np.fft`.  This means that threads can often give really nice performance improvements in Python, while also providing concurrency, despite the GIL.  Of course, Python code that does not block on I/O and that uses computations in native Python, will likely not see much (if any) parallelism when using threads, due to the GIL.

## Parallelism and concurrency with multiprocessing

---

* Advantage: each process has its own GIL, i.e., multiple interpreters that can be accessed in parallel
* Disadvantage: each process has separate memory, must explicitely put things in shared memory, queues or pipes

When the GIL limits our ability to achieve parallelism and we are performing heavy computations in native Python, we may also choose to use the `multiprocessing` module.  Multiprocessing is similar to using threads, except that full-blown heavy-weight processes are spawned instead of lightweight threads.  In fact, each process contains a separate copy of the Python interpreter.  This allows us to overcome the limitations of the GIL since each process can access its own interpreter at the same time.  However, this comes at the cost of having to spawn a full-blown process, which takes time, and also not having fully shared memory between processes.  If we want to send information between processes, we have to set up shared memory, or use shared queues or pipes or sockets in order to for our processes to communicate.  This is in contrast to having the ability to simply modify variables or attributes when using threads.

Below, we see a similar example to the preivous threading example, except that we are now using processes.

In [8]:
# the multiprocessing module is similar to threading but spawns
# full-blown processes, which do not share their entire memory space
import multiprocessing as mp

@timed
def run_processes(n=10):
    # create threads using the previous, nonasync `service_request` function
    processes = [mp.Process(target=service_request, args=(i,)) for i in range(10)]

    # start them all
    for process in processes:
        process.start()

    # wait for them all to finish
    for process in processes:
        process.join()

run_processes()

call 0 iteration 0
call 1 iteration 0
call 2 iteration 0
call 3 iteration 0
call 4 iteration 0
call 5 iteration 0
call 6 iteration 0
call 7 iteration 0
call 8 iteration 0
call 9 iteration 0
end 8 iteration 0
end 9 iteration 0
call 9 iteration 1
end 3 iteration 0
call 8 iteration 1
call 3 iteration 1
end 5 iteration 0
call 5 iteration 1
end 7 iteration 0
call 7 iteration 1
end 6 iteration 0
call 6 iteration 1
end 1 iteration 0
call 1 iteration 1
end 4 iteration 0
end 2 iteration 0
call 4 iteration 1
call 2 iteration 1
end 0 iteration 0
call 0 iteration 1
end 9 iteration 1
call 9 iteration 2
end 5 iteration 1
call 5 iteration 2
end 8 iteration 1
end 3 iteration 1
call 8 iteration 2
call 3 iteration 2
end 1 iteration 1
call 1 iteration 2
end 6 iteration 1
call 6 iteration 2
end 7 iteration 1
call 7 iteration 2
end 0 iteration 1
call 0 iteration 2
end 4 iteration 1
call 4 iteration 2
end 2 iteration 1
call 2 iteration 2
end 9 iteration 2
call 9 iteration 3
end 5 iteration 2
call 5 iteratio

Notice that, in this case, the multiprocessing approach is actually slower than the threading approach!  Again, this is because it takes additional time to fork processes and communicate between processes, while our threading approach already achieves significant parallelism because the GIL is release in the most computationally expensive portions of the code.

## Parallelism and concurrency in C extensions

---

One final, but **very important** thing to note, is that C extensions can do whatever they want, including releasing the GIL or using their own threading model, e.g., pthreads, OpenMP, CUDA, et cetra.  This means that many computational libraries that utilize C extensions under the hood, are often very performant without requiring any sort of threading or multiprocessing on the Python side.  For example, libraries like NumPy, PyTorch and TensorFlow all utilize multiple cores (or even GPUs) without being limited by the GIL.

As a result, it is often appropriate to combine an approach like async in order to provide concurrency and to keep utilization high during blocking requests, with libraries that utilize high-performance threading models from within C extensions.

* Advantage: Extremely flexible
* Disadvantage: Can only be done in C